In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import f1_score
from sklearn import pipeline, preprocessing, feature_extraction
#from lightgbm import LGBMClassifier

In [2]:
import gc
gc.collect()

11

In [3]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

train['is_train'] = np.random.choice([True, False], 135309, p=[0.9, 0.1])

In [11]:
train

,id,url,title,target,is_train
0,0,m.kp.md,"экс-министр экономики молдовы - главе мидэи, ц...",False,False
1,1,www.kp.by,эта песня стала известна многим телезрителям б...,False,True
2,2,fanserials.tv,банши 4 сезон 2 серия бремя красоты смотреть о...,False,True
3,3,colorbox.spb.ru,не беси меня картинки,False,True
4,4,tula-sport.ru,в новомосковске сыграют следж-хоккеисты алекси...,False,True
...,...,...,...,...,...
135304,135304,mail.ru,пора тюльпанов турецкий сериал на русском язык...,False,True
135305,135305,www.ntv.ru,остросюжетный сериал «шеф. игра на повышение»....,False,True
135306,135306,topclassiccarsforsale.com,"1941 plymouth special deluxe hot rod, automati...",False,True
135307,135307,wowcream.ru,купить it's skin сыворотка питательная power 1...,False,True


# Char-based title

In [4]:
train['title'] = train['title'].apply(lambda x: x.lower().split())
test['title'] = test['title'].apply(lambda x: x.lower().split())
#hw = HashingVectorizer(n_features=4000).fit(train['title'])


train['title'] = train['title'].apply(lambda x: ' '.join(x))
test['title'] = test['title'].apply(lambda x: ' '.join(x))

hw = CountVectorizer(max_features=10000, ngram_range=(1,5), analyzer='char').fit(train['title'])


X_char = pd.DataFrame(hw.transform(train['title']).todense())
X_char_test = pd.DataFrame(hw.transform(test['title']).todense())

# Char-based url

In [3]:

hw = CountVectorizer(max_features=150, ngram_range=(1,5), analyzer='char').fit(train['url'])

X_char_url = pd.DataFrame(hw.transform(train['url']).todense())
X_char_url_test = pd.DataFrame(hw.transform(test['url']).todense())

# Word-based title

In [5]:
hw = CountVectorizer(max_features=5000, ngram_range=(1,3)).fit(train['title'])

X_word = pd.DataFrame(hw.transform(train['title']).todense())
X_word_test = pd.DataFrame(hw.transform(test['title']).todense())



# Word-based url

In [6]:
train['url'] = train['url'].apply(lambda x: '-'.join(x.split('.')).split('-'))
test['url'] = test['url'].apply(lambda x: '-'.join(x.split('.')).split('-'))

train['url'] = train['url'].apply(lambda x: ' '.join(x))
test['url'] = test['url'].apply(lambda x: ' '.join(x))

hw = CountVectorizer(max_features=400, ngram_range=(1,3)).fit(train['url'])


X_word_url = pd.DataFrame(hw.transform(train['url']).todense())
X_word_url_test = pd.DataFrame(hw.transform(test['url']).todense())


# concat

In [7]:
X = pd.concat([X_char, X_char_url, X_word, X_word_url], axis=1)
X_test = pd.concat([X_char_test, X_char_url_test, X_word_test, X_word_url_test], axis=1)



In [8]:
del X_char
del X_char_url
del X_char_test
del X_char_url_test
del X_word
del X_word_test
del X_word_url
del X_word_url_test

In [9]:
import gc
gc.collect()

0

In [8]:
reg = LogisticRegression()
#reg = LGBMClassifier()
reg.fit(X[train['is_train'] == True], train[train['is_train'] == True]['target'])

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

# Скор

In [9]:
y_pred = reg.predict(X[train['is_train'] == False])
y_gt = train[train['is_train'] == False]['target'].values
f1_score(y_gt, y_pred)

0.9805471124620061

In [98]:
0.9813200498132005
0.9800747198007472
0.9654320987654322
0.9337194337194337
0.9152433129229777
0.8839960565231679
0.8540145985401459
0.6092814371257484


# test

In [13]:
#reg = LogisticRegression()
reg = LGBMClassifier()

reg.fit(X, train['target'])

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [12]:
test.shape

(165378, 3)

In [14]:
test_pred = reg.predict(X_test)
test['target'] = test_pred

In [15]:
test[['id', 'target']].to_csv("submission_word_char_top_lgb.csv", index=False)

In [1]:
1

1

In [145]:
test

,id,url,title,target
0,135309,www.kommersant.ru,шестой кассационный суд в самаре начнет работу...,False
1,135310,urexpert.online,"что такое индексация алиментов, кем и в каких ...",False
2,135311,imperimeha.ru,женщинам | империя меха - part 12,False
3,135312,national-porn.com,"небритые, волосатые киски: порно всех стран и ...",False
4,135313,2gis.ru,67,False
...,...,...,...,...
165373,300682,etp.armtek.ru,armtek - запчасти для грузовых и легковых авто...,False
165374,300683,mail.ru,"лилия якупова - караганда, карагандинская обла...",False
165375,300684,xn----8sbnqchpeeeth.xn--p1ai,администрация лесного района тверской области ...,False
165376,300685,www-sunhome-ru.cdn.ampproject.org,сонник изменение сознания. к чему снится измен...,False


In [81]:
train

,id,url,title,target,is_train
0,0,m.kp.md,"экс-министр экономики молдовы - главе мидэи, ц...",False,True
1,1,www.kp.by,эта песня стала известна многим телезрителям б...,False,True
2,2,fanserials.tv,банши 4 сезон 2 серия бремя красоты смотреть о...,False,True
3,3,colorbox.spb.ru,не беси меня картинки,False,True
4,4,tula-sport.ru,в новомосковске сыграют следж-хоккеисты алекси...,False,True
...,...,...,...,...,...
135304,135304,mail.ru,пора тюльпанов турецкий сериал на русском язык...,False,True
135305,135305,www.ntv.ru,остросюжетный сериал «шеф. игра на повышение»....,False,True
135306,135306,topclassiccarsforsale.com,"1941 plymouth special deluxe hot rod, automati...",False,True
135307,135307,wowcream.ru,купить it's skin сыворотка питательная power 1...,False,True


In [ ]:
X = pd.DataFrame(hw.transform(train['title']).todense())